In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Wednesday, February 26, 2025
Last Modified: Wednesday, August 13, 2025
Program Purpose: To create a plot which shows all of the individual IAV contribution values across the models.
"""

In [ ]:
#Importing the appropriate libraries
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import glob
import netCDF4 as nc

#Making font times
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block is concerned with the calculation of all of the fluxes across the different models such that they can all be plotted together

# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

# Base K&G Calculations.
all_datasets_IAV = OrderedDict()

# Loading and processing all of the files
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the wrongly high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out 
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Finding the average flux of each grid cell over the 52 years.
    overall_mean_flux_by_grid = terrestrial_flux.mean(axis=0)

    # Finding the flux anomaly for each cell for every year.
    flux_anomaly_yearly_by_grid = terrestrial_flux - overall_mean_flux_by_grid

    # Calculating the yearly global flux anomay by summing all of the individual flux anomalies for each grid cell for that year.
    yearly_global_flux_anomaly = list()
    for i in range(52):
        yearly_global_flux_anomaly.append(np.sum(flux_anomaly_yearly_by_grid[i,:,:]))

    # Finding the bottom sum for the equation (absolute value of global flux anomaly over the years)
    bottom_sum = np.sum(np.abs(yearly_global_flux_anomaly))

    # New array to append top values of the equation to.
    placeholder_array_top_values = np.empty_like(terrestrial_flux)

    for i in range(52):
        # Avoiding division by zero
        if yearly_global_flux_anomaly[i] != 0:
            placeholder_array_top_values[i] = flux_anomaly_yearly_by_grid[i] * (np.abs(yearly_global_flux_anomaly[i]) / yearly_global_flux_anomaly[i])
        else:
            placeholder_array_top_values[i] = 0

    summed_top_values = np.sum(placeholder_array_top_values, axis=0)  # Summing the values for the top of the array
    placeholder_array = summed_top_values / bottom_sum  # Dividing the top and bottom of the equation
    placeholder_array *= 100 ##Multiplying by 100 to make %

    # Adding the placeholder array to the list of IAV maps.
    all_datasets_IAV[file_name] = placeholder_array



In [ ]:
#This block is concerned with the creation of the plot which shows the different IAV contributions of all of the different models.
# Note: With this bar it is too difficult to make out the differences, so the bounds were reduced to -0.01 and 0.05 below

#Masking the coasts with the MODIS data
#Masking out the coastal areas to be aligned with the MODIS data
modis_data = np.load("MODIS_Map.npy")
for model in all_datasets_IAV:
    all_datasets_IAV[model][modis_data == 0] = 0

#Finding the biggest and smallest values
final_max = 0
final_min = 0
for model in all_datasets_IAV:
    #Max
    temp_max= np.nanmax(all_datasets_IAV[model])
    if temp_max > final_max:
        final_max = temp_max
    #Min
    temp_min = np.nanmin(all_datasets_IAV[model])
    if temp_min < final_min:
        final_min = temp_min

norm = mcolors.TwoSlopeNorm(vmin = final_min, vcenter= 0, vmax= final_max) #Setting the colorbar distribution based on the values above

# Create the figure and subplots
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(11, 12.5), subplot_kw={'projection': ccrs.PlateCarree()},)
axes = axes.flatten() #Flattening the axes to more easily cycle through

#Cycling the axes to add the maps to the plot altogether
i = 0
for model in all_datasets_IAV:
    image = axes[i].imshow(all_datasets_IAV[model], norm=norm, cmap='seismic', origin='upper', extent=[-180, 180, -90, 90])
    axes[i].set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded
    #Ading the map features
    axes[i].add_feature(cfeature.BORDERS, linestyle=':')
    axes[i].add_feature(cfeature.COASTLINE)
    # Adding a title to each subplot
    temp_title = str(model).replace("_", "-").upper().replace("-S3", "")
    axes[i].set_title(f"{temp_title}", fontsize=18)
    i+=1


# Adding the colorbar to the left
axes_cbar = fig.add_axes([0,0,0.01,1]) #Location of the cbar
cbar = fig.colorbar(image, cax=axes_cbar) #Adding the colorbar
cbar.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cbar.ax.yaxis.offsetText.set_fontsize(18)  
cbar.ax.yaxis.set_ticks_position('left')
cbar.ax.yaxis.set_label_position('left')
cbar.ax.tick_params(labelsize=18, width=3)
cbar.set_label('IAV Contribution (%)', fontsize=20)

#Tightening up the plot
plt.tight_layout()
plt.show()


In [ ]:
#This block is concerned with the creation of the plot which shows the different IAV contributions of all of the different models.

#These are manually set values based on the findings above
norm = mcolors.TwoSlopeNorm(vmin = -0.01, vcenter= 0, vmax= 0.05) #Setting the colorbar distribution even tighter, such that the regions can be more easily seen

# Create the figure and subplots
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(11, 12.5), subplot_kw={'projection': ccrs.PlateCarree()},)
axes = axes.flatten() #Flattening the axes to more easily cycle through

#Cycling the axes to add the maps to the plot altogether
i = 0
for model in all_datasets_IAV:
    image = axes[i].imshow(all_datasets_IAV[model], norm=norm, cmap='seismic', origin='upper', extent=[-180, 180, -90, 90])
    axes[i].set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded
    #Ading the map features
    axes[i].add_feature(cfeature.BORDERS, linestyle=':')
    axes[i].add_feature(cfeature.COASTLINE)
    # Adding a title to each subplot
    temp_title = str(model).replace("_", "-").upper().replace("-S3", "")
    axes[i].set_title(f"{temp_title}", fontsize=25)
    i+=1


# Adding the colorbar to the left
axes_cbar = fig.add_axes([0,0,0.01,1]) #Location of the cbar
cbar = fig.colorbar(image, cax=axes_cbar) #Adding the colorbar
cbar.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cbar.ax.yaxis.offsetText.set_fontsize(25)  
cbar.ax.yaxis.set_ticks_position('left')
cbar.ax.yaxis.set_label_position('left')
cbar.ax.tick_params(labelsize=25, width=3)
cbar.set_label('IAV Contribution (%)', fontsize=25)

plt.tight_layout()
plt.show()
